In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from tqdm import tqdm
import sys
sys.path.insert(0,"./python")
from analysis_utils import *

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.set(font_scale = 2)
sns.set_style("whitegrid")

In [ ]:
def load_and_calculate_stats(experiment_id):
    experiment_folders = get_experiment_files(experiment_id, snapshots, simulation_dir)
    length_distrib = load_data(experiment_folders, snapshots, "lengths_distrib")
    all_length_distrib = pd.concat(length_distrib)
    all_success = all_length_distrib[all_length_distrib["length"]>0]
    avg_path_length = (all_success["length"]*all_success["count"]).sum() / all_success["count"].sum()
    total_counts = all_length_distrib.groupby("length")["count"].sum()
    S = total_counts.sum()
    total_ratios = total_counts / S
    return avg_path_length, total_ratios

def extract_length_distribution(simulation_dir, experiment_id, snapshots):
    experiment_folders = get_experiment_files(experiment_id, snapshots, simulation_dir)
    length_distrib = load_data(experiment_folders, snapshots, "lengths_distrib")
    all_length_distrib = pd.concat(length_distrib)
    success_transactions = all_length_distrib.copy()[all_length_distrib["length"]>0]
    success_transactions["length"] = success_transactions["length"].apply(lambda x: x if x<6 else "longer")
    return success_transactions.groupby("length")["count"].sum().reset_index()

def generate_pie_chart(image_id, eps, length_distribution, results_dir):
    plt.figure(figsize=(6,6))
    labs = list(length_distribution["length"])
    values = list(length_distribution["count"])
    #plt.pie(values, explode=(0.2,0,0,0,0.2,0.2), labels=[1,2,3,4,5,"longer"], autopct='%1.0f%%', labeldistance=0.7, pctdistance=1.17)
    plt.title("epsilon=%.1f" % eps)
    plt.pie(values, explode=(0.3,0,0,0,0.0,0.3), labels=[1,2,3,4,5,"longer"], autopct='%1.0f%%', labeldistance=1.1, pctdistance=0.7)
    #lgd = plt.legend(labs, loc=2,bbox_to_anchor=(1.08, 1.02))
    #plt.savefig("%s/%s.pdf" % (results_dir, image_id), format="pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.savefig("%s/%s.pdf" % (results_dir, image_id), format="pdf", bbox_inches='tight')

In [ ]:
ratio_col = "ratio of failed transactions"
avg_path_col = "average path length"
tx_num_col = "number of transactions"

# 1. Load path info files

In [ ]:
snapshots = range(7)

In [ ]:
results_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/results/path_length_info/"

### Experiments with capacity filter

simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations_7days/"
amounts = [10000,25000,50000,100000,200000,500000]
tx_nums = [1000,2000,5000,10000,20000]
ts_map = {
    1000:"2019-09-16_20:45:51",
    2000:"2019-09-16_19:13:21",
    5000:"2019-09-16_14:12:54",
    10000:"2019-09-16_17:56:28",
    20000:"2019-09-16_15:39:13"
}

### Experiments without capacity filter

simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/experiments_with_rayleigh/simulations"
amounts = [10000,25000,50000,100000,200000,500000]
tx_nums = [1000,2000,5000,10000,20000,50000]
ts_map = {
    1000:"2019-09-23_22:53:58",
    2000:"2019-09-23_15:29:59",
    5000:"2019-09-23_06:47:40",
    10000:"2019-09-22_22:17:29",
    20000:"2019-09-21_23:53:33",
    50000:"2019-09-23_21:37:58"
}

records = []
for n in tqdm(tx_nums):
    for a in amounts:
        experiment_id = "%s_%isat_k%i" % (ts_map[n], a, n)
        avg_path_length, total_ratios = load_and_calculate_stats(experiment_id)
        ratio_failed = total_ratios[-1]
        records.append([n,a,ratio_failed,avg_path_length])

In [ ]:
simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations_multi/"
amounts = [10000,60000,100000,200000,500000]
tx_nums = [1000,7000,10000,20000,70000]

records = []
for n in tqdm(tx_nums):
    for a in amounts:
        experiment_id = "%isat_k%i_eps0.80" % ( a, n)
        avg_path_length, total_ratios = load_and_calculate_stats(experiment_id)
        ratio_failed = total_ratios[-1]
        records.append([n,a,ratio_failed,avg_path_length])

In [ ]:
path_info_df = pd.DataFrame(records, columns=[tx_num_col,"amount",ratio_col,avg_path_col])

In [ ]:
path_info_df.head()

### Experiments with epsilon

simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations_7days/"
ts = "2019-09-26_14:15:21"
epsilons = [0.0, 0.2, 0.5, 0.8, 1.0]

records = []
for eps in tqdm(epsilons):
        experiment_id = "%s_50000sat_k6000_eps%.2f" % (ts, eps)
        avg_path_length, total_ratios = load_and_calculate_stats(experiment_id)
        ratio_failed = total_ratios[-1]
        records.append([eps,ratio_failed,avg_path_length])

In [ ]:
simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations_multi/"
epsilons = [0.0, 0.2, 0.5, 0.8, 1.0]

records_eps = []
for eps in tqdm(epsilons):
        experiment_id = "60000sat_k7000_eps%.2f" % eps
        avg_path_length, total_ratios = load_and_calculate_stats(experiment_id)
        ratio_failed = total_ratios[-1]
        records_eps.append([eps,ratio_failed,avg_path_length])

In [ ]:
path_info_df_eps = pd.DataFrame(records_eps, columns=["epsilon",ratio_col,avg_path_col])

In [ ]:
path_info_df_eps.head()

# Transaction failure and average path length

## Pivot tables

In [ ]:
pivot_ratio_failed = path_info_df.pivot(index="amount",columns=tx_num_col,values=ratio_col).sort_values("amount",ascending=False)

In [ ]:
pivot_ratio_avg_length = path_info_df.pivot(index="amount",columns=tx_num_col,values=avg_path_col).sort_values("amount",ascending=False)

## Heatmaps

In [ ]:
cmap = "RdBu_r"
fig_s=(8,8)

In [ ]:
fig, ax = plt.subplots(figsize=fig_s)
g = sns.heatmap(data=pivot_ratio_failed, ax=ax, cmap=cmap, square=True, annot=True, fmt='.2f')
g.set_yticklabels(amounts[::-1], rotation=0)
g.set_xticklabels(tx_nums, rotation=90)
ax.set_ylim(6.0, 0)
plt.savefig("%s/ratio_of_failed_paths.pdf" % results_dir, format="pdf", bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=fig_s)
g = sns.heatmap(data=pivot_ratio_avg_length, ax=ax, cmap=cmap,  square=True, annot=True, fmt='.2f')
g.set_yticklabels(amounts[::-1], rotation=0)
g.set_xticklabels(tx_nums, rotation=90)
ax.set_ylim(6.0, 0)
plt.savefig("%s/avg_path_length.pdf" % results_dir, format="pdf", bbox_inches='tight')

# Transaction security analysis

n = 5000
a = 50000
experiment_id = "%s_%isat_k%i" % (ts_map[n], a, n)
length_distrib = extract_length_distribution(simulation_dir, experiment_id, snapshots)
generate_pie_chart("length_distribution_eps%.2f" % eps, length_distrib, results_dir)

for eps in [0.0,0.2,0.5,0.8,1.0]:
    experiment_id = "%s_50000sat_k6000_eps%.2f" % (ts, eps)
    length_distrib = extract_length_distribution(simulation_dir, experiment_id, snapshots)
    generate_pie_chart("length_distribution_eps%.2f" % eps, length_distrib, results_dir)

In [ ]:
for eps in [0.0,0.2,0.5,0.8,1.0]:
    experiment_id = "60000sat_k7000_eps%.2f" % eps
    length_distrib = extract_length_distribution(simulation_dir, experiment_id, snapshots)
    generate_pie_chart("length_distrib_eps%.2f" % eps, eps, length_distrib, results_dir)